# Crypto Screening Strategies

This notebook demonstrates cryptocurrency screening using tvscreener.

**Topics covered:**
- Market cap tiers (large, mid, small cap)
- Momentum strategies
- Technical analysis (RSI, MACD)
- Volume analysis
- Multi-timeframe analysis

In [ ]:
from tvscreener import CryptoScreener, CoinScreener
from tvscreener.field import CryptoField, CoinField
import pandas as pd

---
## Basic Crypto Screening

In [ ]:
cs = CryptoScreener()
df = cs.get()
print(f"Retrieved {len(df)} cryptos with {len(df.columns)} columns")
df.head(10)

---
## Market Cap Tiers

### Large Cap (>$10B)

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.MARKET_CAPITALIZATION > 10e9)  # >$10B

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.MARKET_CAPITALIZATION,
    CryptoField.CHANGE_PERCENT,
    CryptoField.VOLUME
)
cs.sort_by(CryptoField.MARKET_CAPITALIZATION, ascending=False)

df = cs.get()
print(f"Found {len(df)} large cap cryptos (>$10B)")
df.head(15)

### Mid Cap ($100M - $1B)

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.MARKET_CAPITALIZATION.between(100e6, 1e9))
cs.where(CryptoField.VOLUME > 5_000_000)  # Liquidity filter

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.MARKET_CAPITALIZATION,
    CryptoField.CHANGE_PERCENT,
    CryptoField.VOLUME
)
cs.sort_by(CryptoField.CHANGE_PERCENT, ascending=False)

df = cs.get()
print(f"Found {len(df)} mid cap cryptos ($100M-$1B)")
df.head(15)

---
## Momentum Strategies

### Daily Top Gainers

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.CHANGE_PERCENT > 10)  # Up 10%+
cs.where(CryptoField.VOLUME > 10_000_000)  # Minimum volume

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.CHANGE_PERCENT,
    CryptoField.VOLUME,
    CryptoField.MARKET_CAPITALIZATION
)
cs.sort_by(CryptoField.CHANGE_PERCENT, ascending=False)
cs.set_range(0, 50)

df = cs.get()
print(f"Top {len(df)} crypto gainers today")
df.head(15)

### Weekly Momentum

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.PERFORMANCE_1_WEEK > 20)  # Up 20%+ weekly
cs.where(CryptoField.MARKET_CAPITALIZATION > 100e6)

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.CHANGE_PERCENT,
    CryptoField.PERFORMANCE_1_WEEK,
    CryptoField.VOLUME
)
cs.sort_by(CryptoField.PERFORMANCE_1_WEEK, ascending=False)

df = cs.get()
print(f"Found {len(df)} cryptos with strong weekly momentum")
df.head(15)

---
## Technical Analysis

### RSI Oversold

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.RELATIVE_STRENGTH_INDEX_14 < 30)  # Oversold
cs.where(CryptoField.MARKET_CAPITALIZATION > 100e6)  # Established projects

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.RELATIVE_STRENGTH_INDEX_14,
    CryptoField.CHANGE_PERCENT,
    CryptoField.MARKET_CAPITALIZATION
)
cs.sort_by(CryptoField.RELATIVE_STRENGTH_INDEX_14, ascending=True)

df = cs.get()
print(f"Found {len(df)} oversold cryptos (RSI < 30)")
df.head(15)

### MACD Bullish

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.MACD_LEVEL_12_26 > 0)  # MACD positive
cs.where(CryptoField.MACD_LEVEL_12_26 > CryptoField.MACD_SIGNAL_12_26_9)  # Above signal
cs.where(CryptoField.VOLUME > 5_000_000)

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.MACD_LEVEL_12_26,
    CryptoField.MACD_SIGNAL_12_26_9,
    CryptoField.CHANGE_PERCENT
)

df = cs.get()
print(f"Found {len(df)} cryptos with bullish MACD")
df.head(15)

### Above Moving Averages

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.PRICE > CryptoField.SIMPLE_MOVING_AVERAGE_50)
cs.where(CryptoField.SIMPLE_MOVING_AVERAGE_50 > CryptoField.SIMPLE_MOVING_AVERAGE_200)
cs.where(CryptoField.MARKET_CAPITALIZATION > 1e9)

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.SIMPLE_MOVING_AVERAGE_50,
    CryptoField.SIMPLE_MOVING_AVERAGE_200,
    CryptoField.CHANGE_PERCENT
)

df = cs.get()
print(f"Found {len(df)} cryptos with bullish MA alignment")
df.head(15)

### Multi-Timeframe RSI

In [ ]:
cs = CryptoScreener()

# Daily RSI moderate
cs.where(CryptoField.RELATIVE_STRENGTH_INDEX_14.between(35, 55))

# 4-hour RSI oversold
rsi_4h = CryptoField.RELATIVE_STRENGTH_INDEX_14.with_interval('240')
cs.where(rsi_4h < 35)

cs.where(CryptoField.MARKET_CAPITALIZATION > 500e6)

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.RELATIVE_STRENGTH_INDEX_14,
    rsi_4h,
    CryptoField.CHANGE_PERCENT
)

df = cs.get()
print(f"Found {len(df)} cryptos: daily RSI neutral, 4h RSI oversold")
df.head(15)

---
## Volume Analysis

### High Volume Movers

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.VOLUME > 100_000_000)  # $100M+ daily volume
cs.where(CryptoField.CHANGE_PERCENT.not_between(-2, 2))  # Significant move

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.CHANGE_PERCENT,
    CryptoField.VOLUME,
    CryptoField.MARKET_CAPITALIZATION
)
cs.sort_by(CryptoField.VOLUME, ascending=False)

df = cs.get()
print(f"Found {len(df)} high volume movers")
df.head(15)

---
## Volatility Screens

### Low Volatility Large Cap

In [ ]:
cs = CryptoScreener()

cs.where(CryptoField.MARKET_CAPITALIZATION > 5e9)
cs.where(CryptoField.VOLATILITY_DAY < 3)  # Low daily volatility

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.VOLATILITY_DAY,
    CryptoField.CHANGE_PERCENT,
    CryptoField.MARKET_CAPITALIZATION
)
cs.sort_by(CryptoField.MARKET_CAPITALIZATION, ascending=False)

df = cs.get()
print(f"Found {len(df)} low volatility large cap cryptos")
df.head(15)

---
## Specific Cryptocurrencies

In [ ]:
cs = CryptoScreener()

# Specify tickers
cs.symbols = {
    "query": {"types": []},
    "tickers": [
        "BINANCE:BTCUSDT",
        "BINANCE:ETHUSDT",
        "BINANCE:SOLUSDT",
        "BINANCE:ADAUSDT",
        "BINANCE:DOTUSDT"
    ]
}

cs.select(
    CryptoField.NAME,
    CryptoField.PRICE,
    CryptoField.CHANGE_PERCENT,
    CryptoField.VOLUME,
    CryptoField.RELATIVE_STRENGTH_INDEX_14
)

df = cs.get()
df

---
## Coin Screener (CoinGecko data)

Alternative screener with different data source.

In [ ]:
coins = CoinScreener()

coins.sort_by(CoinField.MARKET_CAPITALIZATION, ascending=False)
coins.set_range(0, 100)

coins.select(
    CoinField.NAME,
    CoinField.PRICE,
    CoinField.MARKET_CAPITALIZATION,
    CoinField.CHANGE_PERCENT
)

df = coins.get()
print(f"Top {len(df)} coins by market cap")
df.head(15)

---
## Select All Fields

In [ ]:
cs = CryptoScreener()
cs.select_all()
cs.set_range(0, 10)  # Limit for demo

df = cs.get()
print(f"Retrieved {len(df.columns)} columns for {len(df)} cryptos")
df.head()